In [3]:
from sys import path
path.append('../')

In [4]:
from core.request import FactoryClient

In [5]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [6]:
import nest_asyncio
nest_asyncio.apply()

In [7]:
test_address = '0x7e5ce10826ee167de897d262fcc9976f609ecd2b'

In [8]:

# result = f.sync_request(
#     namespace='/address',
#     payload={
#         'address': test_address,
#         'currency': 'usd',
#         'portfolio_fields': 'all',
# #         'transactions_offset': 2
#     },
#     scope='transactions'
# )

In [12]:
def parce_transactions(wallet_hash):
    f = FactoryClient('../settings.yaml')
    dicts = []
    i = 50
    data = f.sync_request(
        namespace='/address',
        payload={
            'address': wallet_hash,
            'currency': 'usd',
            'portfolio_fields': 'all',
            'transactions_offset': 0
        },
        scope='transactions'
    )
    dicts.append(data)

    while len(data) > 0:
        f = FactoryClient('../settings.yaml')
        data = f.sync_request(
                                namespace='/address',
                                payload={
                                    'address': test_address,
                                    'currency': 'usd',
                                    'portfolio_fields': 'all',
                                    'transactions_offset': i
                                        },
                                scope='transactions'
                                )
        i += 50
        dicts.append(data)
    return dicts

In [13]:
dicts = parce_transactions(test_address)

Connected to /address namespace!
Address portfolio is received
Connected to /address namespace!
Address portfolio is received
Connected to /address namespace!
Address portfolio is received
Connected to /address namespace!
Address portfolio is received
Connected to /address namespace!
Address portfolio is received
Connected to /address namespace!
Address portfolio is received
Connected to /address namespace!
Address portfolio is received
Connected to /address namespace!
Address portfolio is received
Connected to /address namespace!
Address portfolio is received
Connected to /address namespace!
Address portfolio is received
Connected to /address namespace!
Address portfolio is received
Connected to /address namespace!
Address portfolio is received
Connected to /address namespace!
Address portfolio is received
Connected to /address namespace!
Address portfolio is received
Connected to /address namespace!
Address portfolio is received
Connected to /address namespace!
Address portfolio is r

In [31]:
dicts[0][0]

{'id': '827f8b05-fb664ffdaac7101f-7e5ce108',
 'type': 'send',
 'protocol': None,
 'mined_at': 1623409428,
 'block_number': 12612869,
 'status': 'confirmed',
 'hash': '0xfb664ffdaac7101f2b751eb5e3c45e60a4dc52431d9f79cecb85c4c2b7781ff2',
 'direction': 'out',
 'address_from': '0x7e5ce10826ee167de897d262fcc9976f609ecd2b',
 'address_to': '0x51020400c2aa3ed9856435716c50b6ef2f38e084',
 'contract': None,
 'nonce': 660,
 'changes': [{'asset': {'asset_code': 'eth',
    'name': 'Ethereum',
    'symbol': 'ETH',
    'decimals': 18,
    'type': None,
    'icon_url': 'https://s3.amazonaws.com/token-icons/eth.png',
    'price': {'value': 2408.05,
     'relative_change_24h': -0.843308503497997,
     'changed_at': 0},
    'is_displayable': True,
    'is_verified': True},
   'value': 500000000000000000,
   'direction': 'out',
   'address_from': '0x7e5ce10826ee167de897d262fcc9976f609ecd2b',
   'address_to': '0x51020400c2aa3ed9856435716c50b6ef2f38e084',
   'price': 2485.25}],
 'fee': {'value': 105000000000

In [111]:
def get_price(changes_dict, deg=18):
    stock_price = changes_dict['price']
    value = changes_dict['value']
    if (stock_price is None) or (value is None):
        return np.nan
    order_price = (value / 10**deg) * stock_price
    return order_price

def income_outcome(changes):
    income = 0
    outcome = 0
    for change in changes:
        deg = change['asset']['decimals']
        if change['direction'] == 'in':
            income += get_price(change, deg)
        elif change['direction'] == 'out':
            outcome += get_price(change, deg)
    return income, outcome

def parse_transaction(transaction, type_of_deal, mined_at, hash_tr):
    if 'changes' in transaction:
        changes = transaction['changes']
    else: 
        return None
    
    income, outcome = income_outcome(changes)
        
    fee = transaction['fee']
    if fee is not None:
        fee_price = get_price(fee)
    else:
        fee_price = np.nan
        
    df = pd.DataFrame(data={'timestamp': mined_at,
                            'type': type_of_deal,
                            'from': transaction['address_from'],
                            'to': transaction['address_to'],
                            'income': income, 
                            'outcome': outcome, 
                            'fee': fee_price}, 
                     index=[hash_tr])
    return df


In [112]:
dfsss = []
for i, data in enumerate(dicts[:-1]):
    print(i)
    dfs = []
    for res in tqdm(data):
        if res['type'] in ['send', 'receive', 'trade',
                           'claim', 'authorize', 'withdraw',
                           'deposit', 'borrow', 'repay',
                          'stake', 'unstake', 'deployment',
                          'execution']:
            
            chank = parse_transaction(res, res['type'], res['mined_at'], res['hash'])
            dfs.append(chank)
        else:
            print(res['type'])
    dfsss.append(pd.concat(dfs))

  0%|          | 0/50 [00:00<?, ?it/s]

0
1
2


100%|██████████| 50/50 [00:00<00:00, 1912.73it/s]

3
4
5
6
7



100%|██████████| 50/50 [00:00<00:00, 1688.91it/s]


8
9
10
11
12


100%|██████████| 28/28 [00:00<00:00, 1763.32it/s]

13
14
15


In [114]:
pd.concat(dfsss).to_csv('exaple_data.csv')

In [93]:
dicts

[[{'id': '827f8b05-fb664ffdaac7101f-7e5ce108',
   'type': 'send',
   'protocol': None,
   'mined_at': 1623409428,
   'block_number': 12612869,
   'status': 'confirmed',
   'hash': '0xfb664ffdaac7101f2b751eb5e3c45e60a4dc52431d9f79cecb85c4c2b7781ff2',
   'direction': 'out',
   'address_from': '0x7e5ce10826ee167de897d262fcc9976f609ecd2b',
   'address_to': '0x51020400c2aa3ed9856435716c50b6ef2f38e084',
   'contract': None,
   'nonce': 660,
   'changes': [{'asset': {'asset_code': 'eth',
      'name': 'Ethereum',
      'symbol': 'ETH',
      'decimals': 18,
      'type': None,
      'icon_url': 'https://s3.amazonaws.com/token-icons/eth.png',
      'price': {'value': 2408.05,
       'relative_change_24h': -0.843308503497997,
       'changed_at': 0},
      'is_displayable': True,
      'is_verified': True},
     'value': 500000000000000000,
     'direction': 'out',
     'address_from': '0x7e5ce10826ee167de897d262fcc9976f609ecd2b',
     'address_to': '0x51020400c2aa3ed9856435716c50b6ef2f38e084',

In [ ]:



"borrow"

"repay"

"stake"

"unstake"

